In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Binarizer, MultiLabelBinarizer

import tensorflow as tf
# from tensorflow.keras.layers import Dropout, Flatten, Dense, InputLayer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback, ReduceLROnPlateau
# from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.backend import clear_session

import keras
from keras import backend as K
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam

### model py files
import CNN_BatchN as batchN

### E. CNN_Baseline + Haze Removal + Data Augmentation + Batch Normalization + Drop out + Early Stopping

In [2]:
# Import dataset
DATASET_PATH = 'dataset/'
TRAIN_PATH = DATASET_PATH + 'train_file'
TEST_PATH = DATASET_PATH + 'test_file'

TRAIN_CSV_PATH = DATASET_PATH + 'train_label.csv'
TEST_CSV_PATH = DATASET_PATH + 'test_label.csv'

df_train = pd.read_csv(TRAIN_CSV_PATH)
df_test = pd.read_csv(TEST_CSV_PATH)

df_train['image_name'] = df_train['image_name'].astype(str)
df_test['image_name'] = df_test['image_name'].astype(str)

df_train['tags'] = df_train['tags'].apply(lambda x: x.split(' '))
df_test['tags'] = df_test['tags'].apply(lambda x: x.split(' '))

X_train_files_o = np.array(df_train['image_name'].tolist()) # filenames
X_train_files_o.reshape((X_train_files_o.shape[0], 1))
y_train_o = np.array(df_train['tags'].tolist(), dtype=object) # train image tags (ground truth)
y_test_o = np.array(df_test['tags'].tolist(), dtype=object) # test image tags (ground truth)

tags = df_train['tags'].values
flat_list = [item for sublist in tags for item in sublist]
tags_unique, tags_count = np.unique(flat_list, return_counts=True)
labels = list(tags_unique)

In [3]:
# Define learning curve and evaluation metric (f-2 Beta score):

def learning_curve(model_fit, key='acc', ylim=(0.8, 1.01)):
    plt.figure(figsize=(12,6))
    plt.plot(model_fit.history[key])
    plt.plot(model_fit.history['val_' + key])
    plt.title('Learning Curve')
    plt.ylabel(key.title())
    plt.xlabel('Epoch')
    plt.ylim(ylim)
    plt.legend(['train', 'test'], loc='best')
    plt.show()

# F-Beta Score
def fbeta_score_K(y_true, y_pred):
    beta = 2
    beta_squared = beta ** 2
    tp = K.sum(y_true * y_pred) + K.epsilon()
    fp = K.sum(y_pred) - tp
    fn = K.sum(y_true) - tp

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    result = (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())
    return result

from sklearn.metrics import fbeta_score


def calc_acc(y_test, y_pred, labels=labels, threshold=0.2):
    
    array_labels = np.array(labels)
    test = y_test
    pred = y_pred
    
    # Binarize pred to 0 and 1...
    binarizer = Binarizer(threshold=threshold)
    pred = binarizer.fit_transform(pred)
    # 0 and 1 to False and True for boolean indexing...
    pred = pred > 0
    
    preds = []
    for row in pred:
        preds.append(array_labels[row])
    
    mlb = MultiLabelBinarizer(classes=array_labels)
    
    test = mlb.fit_transform(test)
    preds = mlb.transform(preds)
    score = fbeta_score(test, preds, beta=2, average='weighted')
    
    return score


def plot(result, csv_filename):
    train_history = pd.DataFrame(result)

    train_history.to_csv(csv_filename, sep=',')
    plt.plot(train_history['fbeta_score_K'])
    plt.plot(train_history['val_fbeta_score_K'])
    plt.title('model accuracy')
    plt.ylabel('fbeta_score_K')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

    plt.plot(train_history['loss'])
    plt.plot(train_history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [4]:
# data_generator for keras CNN
def data_generator(X_train_files_o, y_train_o, base_flg, batch_size):
    
    X_train_files, X_val_files, y_train, y_val = train_test_split(X_train_files_o, y_train_o, test_size=0.2, random_state=0)

    train_df = pd.DataFrame(list(zip(X_train_files, y_train)), columns = ['image_name', 'tags'])
    val_df = pd.DataFrame(list(zip(X_val_files, y_val)), columns = ['image_name', 'tags'])

    train_df['tags'] = train_df['tags']
    val_df['tags'] = val_df['tags']
    
    if base_flg:
        train_datagen = ImageDataGenerator(
            rescale=1./255
            )
    else:
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
            )
        
    train_generator = train_datagen.flow_from_dataframe(
        train_df,
        directory=TRAIN_PATH,
        x_col='image_name',
        y_col='tags',
        target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
        class_mode='categorical',
        batch_size=batch_size,
        classes=labels,
        )

    val_datagen = ImageDataGenerator(
        rescale=1./255
        )

    val_generator = val_datagen.flow_from_dataframe(
        val_df,
        directory=TRAIN_PATH,
        x_col='image_name',
        y_col='tags',
        target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
        class_mode='categorical',
        batch_size=batch_size,
        classes=labels,
        )

    test_datagen = ImageDataGenerator(
        rescale=1./255
        )

    test_generator = test_datagen.flow_from_dataframe(
        df_test,
        directory=TEST_PATH,
        x_col='image_name',
        y_col='tags',
        target_size=(INPUT_SHAPE[0], INPUT_SHAPE[1]),
        class_mode='categorical',
        batch_size=BATCH_SIZE,
        classes=labels,
        shuffle=False,
        )
    
    return train_generator,val_generator,test_generator

In [5]:
# Parameters
INPUT_SHAPE = (128, 128, 3) # Image Dimensions
EPOCHS = 100
REG_STRENGTH = 0.01 # Regularization Strength
WORKERS = 4 # Multithreading no of threads
MAXQ = 10 # Max Queue size for multithreading
THRES = [0.2] * 17 # Threshold for truth value of label, applied on sigmoid output.

### Hyperparameter grid
BATCH_SIZE = [32, 64, 128]
DROPOUT_RATE = [0.1, 0.25, 0.5]
PATIENCE = [5, 10, 20]
LR = [0.0001, 0.001, 0.01] # Learning Rate

In [6]:
### Import model
def run(BATCH_SIZE, DROPOUT_RATE, PATIENCE, LR):
    results = {} # record hyperparameter sets, val_loss and val_fbeta_score_K here

    for batch_size in BATCH_SIZE:
        print('batch_size =', batch_size)
        for dropout_rate in DROPOUT_RATE:
            print('\tdropout_rate =', dropout_rate)
            for patience in PATIENCE:
                print('\t\tpatience =', patience)
                for lr in LR:
                    print('\t\t\tlr =', lr)
                    train_generator, val_generator, test_generator = data_generator(X_train_files_o, y_train_o, False, batch_size)
                    model_path = 'CNN_last_weights' + '.h5'
                    adam = Adam(learning_rate=lr)
                    cnn_batch_model = batchN.create_model(dropout_rate)
                    clear_session() # tf backend
                    cnn_batch_model.compile(loss='binary_crossentropy', optimizer=adam, metrics=[fbeta_score_K])

                    callbacks = [
                        ModelCheckpoint(model_path, monitor='val_fbeta_score_K', save_best_only=True, mode='max'),
                        ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3, mode='min', min_lr=0.000001),
                        EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience),
                    ]

                    history = cnn_batch_model.fit(train_generator, epochs=EPOCHS, validation_data=val_generator, callbacks=callbacks,
                                       workers=WORKERS, use_multiprocessing=False, max_queue_size=MAXQ)
                    
                    print(history.history)

#                     cnn_batch_model.load_weights(model_path)

#                     y_pred_hr_da_do_es_bn = cnn_batch_model.predict(test_generator, workers=WORKERS, use_multiprocessing=False, max_queue_size=MAXQ)

    return results

In [7]:
results = run(BATCH_SIZE, DROPOUT_RATE, PATIENCE, LR)

batch_size = 32
	dropout_rate = 0.1
		patience = 5
			lr = 0.0001
Found 25906 validated image filenames belonging to 17 classes.
Found 6477 validated image filenames belonging to 17 classes.
Found 8096 validated image filenames belonging to 17 classes.
Epoch 1/100
205/810 [======>.......................] - ETA: 41s - loss: 0.2656 - fbeta_score_K: 0.5910

KeyboardInterrupt: 

In [ ]:
### Iterate each set of hyperparameters and select the one with lowest loss

### XGBoost